## Zadanie

Dla danych `pima_diabetes.csv` zbuduj jak najlepszy model klasyfikacji, który wykrywa początek cukrzycy u pacjentów. W tym celu wykorzystaj 2 metody: SVM z radialną funkcją bazową oraz inny (dowolny) klasyfikator poznany na zajęciach (drzewo decyzyjne, kNN, regresja logistyczna). Dla każdej wybranej metody klasyfikacji spróbuj dobrać parametry klasyfikatora w taki sposób aby uzyskać jak najlepszą generalizację (tj. poprawność na zbiorze testowym).  


### Dane: Pima indian diabetes

Zbiór danych znajduje się w katalogu [dane/pima_diabetes.csv](dane/pima_diabetes.csv).

Plik zawiera pomiary 600 pacjentów (kobiety w wieku powyżej 21 lat pochodzące z grupy indian Pima) zapisanych w formacie CSV. Wartości oddzielone przecinkiem, nazwy zmiennych zą zawarte w pierwszej linii pliku. 

Informacje o zmiennych: 
1. `Pregnancies` - liczba ciąży 
2. `Glucose` - stężenie glukozy w osoczu
3. `BloodPressure` - rozkurczowe ciśnienie krwi (mm Hg) 
4. `SkinThickness` - grubość fałdu skórnego w tricepsie (mm) 
5. `Insulin` - insulina w surowicy (mu U/ml) 
6. `BMI` - wskaźnik masy ciała 
7. `DiabetesPedigreeFunction` - ocenia prawdopodobieństwo cukrzycy na podstawie wywiadu rodzinnego 
8. `Age`: wiek (lata) 
9. `Outcome `: klasa (0 lub 1)


1. Zaimportuj zbiór danych i podziel go na część treningową i testową.  
Jeżeli uznasz to za stosowane, to możesz przetransformować dane treningowe w dowolny sposób w celu uzyskania wyższej poprawności klasyfikacji, np.: standaryzacja, normalizacja, usunięcie wartości odstających, usunięcie nieistotnych zmiennych, usunięcie wartości brakujących, itp. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


df = pd.read_csv('dane/pima_diabetes.csv')

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

zero_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
X_train[zero_cols] = X_train[zero_cols].replace(0, np.nan)
X_test[zero_cols] = X_test[zero_cols].replace(0, np.nan)

imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)


2. Zbuduj model klasyfikacji o jak najlepszym poziomie poprawności rozpoznawania wczesnej cukrzycy, czyli model predykcji zmiennej `Outcome`, która przyjmuje wartości 0 lub 1.   
Wykorzystaj 2 metody klasyfikacji:
   * SVM z radialną funkcją bazową `rbf`. Dobierz najlepszą wartość parametru regularyzacji `C` oraz najlepszą wartość parametru `gamma`, który odpowiada za szerokość użytej funkcji radialnej (wielkość odchylenia funkcji Gaussa).
   * inna (dowolna) metoda z zajęć: drzewo decyzyjne, kNN lub regresja logistyczna.

Wyraźnie zaznacz (np. w komentarzu), który z uzyskanych modeli klasyfikacji (rodzaj metody i konfiguracja parametrów) uznajesz za najlepszy dla tych danych.

In [2]:
# Metoda 1
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svm_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']
}

svm_grid = GridSearchCV(SVC(kernel='rbf', random_state=42), svm_params, cv=5, scoring='accuracy')
svm_grid.fit(X_train_scaled, y_train)

# Best SVM model
best_svm = svm_grid.best_estimator_
svm_pred = best_svm.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)

print(f"SVM najlepsze parametry: {svm_grid.best_params_}")
print(f"SVM dokładność: {svm_accuracy:.4f}")

SVM najlepsze parametry: {'C': 100, 'gamma': 0.01}
SVM dokładność: 0.7583


In [3]:
# Metoda 2

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)
log_reg_pred = log_reg.predict(X_test_scaled)
log_reg_accuracy = accuracy_score(y_test, log_reg_pred)

print(f"Regresja logistyczna dokładność: {log_reg_accuracy:.4f}")

Regresja logistyczna dokładność: 0.7583


In [4]:
from sklearn.tree import DecisionTreeClassifier

# Metoda 3

tree_params = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

tree_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), tree_params, cv=5, scoring='accuracy')
tree_grid.fit(X_train_scaled, y_train)

best_tree = tree_grid.best_estimator_
tree_pred = best_tree.predict(X_test_scaled)
tree_accuracy = accuracy_score(y_test, tree_pred)

print(f"Drzewo decyzyjne najlepsze parametry: {tree_grid.best_params_}")
print(f"Drzewo decyzyjne dokładność: {tree_accuracy:.4f}")

Drzewo decyzyjne najlepsze parametry: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
Drzewo decyzyjne dokładność: 0.7417


In [5]:
from sklearn.neighbors import KNeighborsClassifier

# Metoda 4

knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21]
}

knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy')
knn_grid.fit(X_train_scaled, y_train)

best_knn = knn_grid.best_estimator_
knn_pred = best_knn.predict(X_test_scaled)
knn_accuracy = accuracy_score(y_test, knn_pred)

print(f"kNN najlepsze parametry: {knn_grid.best_params_}")
print(f"kNN dokładność: {knn_accuracy:.4f}")

kNN najlepsze parametry: {'n_neighbors': 15}
kNN dokładność: 0.7667


In [6]:
# WYBÓR NAJLEPSZEGO MODELU
accuracies = {
    'SVM': (svm_accuracy, f"C={svm_grid.best_params_['C']}, gamma={svm_grid.best_params_['gamma']}"),
    'Drzewo decyzyjne': (tree_accuracy, f"max_depth={tree_grid.best_params_['max_depth']}, min_samples_split={tree_grid.best_params_['min_samples_split']}, min_samples_leaf={tree_grid.best_params_['min_samples_leaf']}"),
    'kNN': (knn_accuracy, f"n_neighbors={knn_grid.best_params_['n_neighbors']}")
}

best_model_name = max(accuracies, key=lambda x: accuracies[x][0])
best_accuracy, best_params = accuracies[best_model_name]

print(f"NAJLEPSZY MODEL: {best_model_name} z parametrami: {best_params}")
print(f"Dokładność: {best_accuracy:.4f}")

NAJLEPSZY MODEL: kNN z parametrami: n_neighbors=15
Dokładność: 0.7667
